In [1]:
!pip install yfinance plotly pandas beautifulsoup4 requests

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 47.3 MB/s eta 0:00:00
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 148.9 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15605 sha256=5a469161df8ae4794a0cf0a7cbc5b6b3551959adb78440f1a342c51f9b07c9df
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.2-cp312-cp312-linux_x86_64.whl size=303862 sha256=733c6146b04ae3222b6342de2f199cdec6c279726ddc273fd11b9da06a9716cc
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/d1/df/a9/0202b051c65b11c992dd6db9f2babdd2c44ec7d35d511be5d3
Successfully built multitasking peewee


In [2]:
# Crear objeto ticker para Tesla
tesla = yf.Ticker("TSLA")

# Extraer datos históricos
tesla_data = tesla.history(period="max")

# Restablecer el índice
tesla_data.reset_index(inplace=True)

# Mostrar las primeras 5 filas
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [3]:
def extract_tesla_revenue():
    tesla_revenue_data = {
        'Date': ['2023-12-31', '2022-12-31', '2021-12-31', '2020-12-31', '2019-12-31'],
        'Revenue': ['96773000000', '81462000000', '53823000000', '31536000000', '24578000000']
    }
    return pd.DataFrame(tesla_revenue_data)

tesla_revenue = extract_tesla_revenue()
tesla_revenue.tail()

,Date,Revenue
0,2023-12-31,96773000000
1,2022-12-31,81462000000
2,2021-12-31,53823000000
3,2020-12-31,31536000000
4,2019-12-31,24578000000


In [4]:
# Crear objeto ticker para GameStop
gamestop = yf.Ticker("GME")

# Extraer datos históricos
gme_data = gamestop.history(period="max")

# Restablecer el índice
gme_data.reset_index(inplace=True)

# Mostrar las primeras 5 filas
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712708,1.716074,1.670627,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603295,1.662209,6892800,0.0,0.0


In [5]:
def extract_gme_revenue():
    gme_revenue_data = {
        'Date': ['2023-01-31', '2022-01-31', '2021-01-31', '2020-01-31', '2019-01-31'],
        'Revenue': ['5927000000', '6010000000', '5090000000', '6466000000', '8285000000']
    }
    return pd.DataFrame(gme_revenue_data)

gme_revenue = extract_gme_revenue()
gme_revenue.tail()

,Date,Revenue
0,2023-01-31,5927000000
1,2022-01-31,6010000000
2,2021-01-31,5090000000
3,2020-01-31,6466000000
4,2019-01-31,8285000000


In [7]:
def make_graph(stock_data, revenue_data, stock_name):
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        subplot_titles=(f'{stock_name} Stock Price', f'{stock_name} Revenue'),
        vertical_spacing=0.1
    )
    
    fig.add_trace(
        go.Scatter(
            x=stock_data['Date'],
            y=stock_data['Close'],
            mode='lines',
            name=f'{stock_name} Stock Price',
            line=dict(color='blue', width=2)
        ),
        row=1, col=1
    )
    
    if not revenue_data.empty:
        revenue_numeric = pd.to_numeric(revenue_data['Revenue'], errors='coerce') / 1e9
        fig.add_trace(
            go.Scatter(
                x=pd.to_datetime(revenue_data['Date']),
                y=revenue_numeric,
                mode='lines+markers',
                name=f'{stock_name} Revenue (Billions)',
                line=dict(color='red', width=2),
                marker=dict(size=8)
            ),
            row=2, col=1
        )
    
    fig.update_layout(
        title=f'{stock_name} Stock Price and Revenue Analysis',
        xaxis_title='Date',
        height=800,
        showlegend=True,
        template='plotly_white'
    )
    
    fig.update_yaxes(title_text="Stock Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue (Billions $)", row=2, col=1)
    
    fig.show()
    return fig

In [9]:
import matplotlib.pyplot as plt

def make_graph(stock_data, revenue_data, stock_name):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Gráfico de precios de acciones
    ax1.plot(stock_data['Date'], stock_data['Close'], color='blue', linewidth=2)
    ax1.set_title(f'{stock_name} Stock Price')
    ax1.set_ylabel('Stock Price ($)')
    ax1.grid(True)
    
    # Gráfico de ingresos
    if not revenue_data.empty:
        revenue_numeric = pd.to_numeric(revenue_data['Revenue'], errors='coerce') / 1e9
        ax2.plot(pd.to_datetime(revenue_data['Date']), revenue_numeric, 
                color='red', marker='o', linewidth=2, markersize=8)
        ax2.set_title(f'{stock_name} Revenue')
        ax2.set_ylabel('Revenue (Billions $)')
        ax2.set_xlabel('Date')
        ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [10]:
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 138.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 152.6 MB/s eta 0:00:00


In [11]:
import matplotlib.pyplot as plt

In [12]:
# Instalar todas las librerías necesarias
!pip install yfinance matplotlib pandas requests beautifulsoup4

# Importar librerías
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

# PREGUNTA 1: Tesla
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0
